Будем использовать физическими величинами с измерениями. Модуль PhysicalQuantities сломался, не обновлялся два года, поэтому придется попробовать новое.

In [6]:
from quantiphy import Quantity as Q

ImportError: cannot import name 'StatelessInputTransformer' from 'IPython.core.inputtransformer2' (/home/artur/atmoescape/.venv/lib64/python3.12/site-packages/IPython/core/inputtransformer2.py)